```{contents}
```
## Cancel

In LangGraph, **cancel** is a **runtime control mechanism** that **forcibly stops an executing graph instance** and safely terminates all in-flight nodes, tools, and state transitions.
It is a core feature for building **responsive, safe, and production-grade** LLM systems.

---

### **1. Why Cancel Exists**

Long-running LLM workflows can:

* Hang on slow tools
* Enter unintended loops
* Become irrelevant due to new user input
* Exceed budgets
* Require immediate shutdown for safety

**Cancel provides deterministic termination** of a live execution.

---

### **2. Conceptual Model**

```
Graph Execution
   |
   |----> Node A (running)
   |----> Node B (waiting)
   |----> Node C (scheduled)
   |
[CANCEL SIGNAL]
   |
Execution Engine stops all work
→ state checkpointed
→ resources released
→ execution ends
```

Cancel is **external to the graph logic** and operates at the **runtime layer**.

---

### **3. Cancellation Lifecycle**

| Phase            | Description                              |
| ---------------- | ---------------------------------------- |
| Issue Cancel     | Client / system sends cancel signal      |
| Propagation      | Runtime propagates cancellation to nodes |
| Node Termination | Active nodes receive stop signal         |
| Cleanup          | Resources, tools, connections closed     |
| Checkpoint       | Final state snapshot saved               |
| Termination      | Execution marked as cancelled            |

---

### **4. How Cancellation Works Internally**

LangGraph uses **cooperative cancellation**:

* Each node execution is wrapped in a cancellable task
* Cancel triggers:

  * Task interruption
  * Future scheduling halt
  * State mutation freeze

---

### **5. Using Cancel in Practice**

### **A. Cancel from Client (Async Execution)**

```python
run = graph.astream(input_data)

# later
await run.cancel()
```

---

### **B. Timeout-Based Auto Cancel**

```python
result = graph.invoke(input_data, config={"timeout": 30})
```

If execution exceeds 30 seconds → **automatic cancellation**

---

### **C. User-Driven Cancel (UI / API)**

```python
run_id = graph.invoke_async(input_data)

# user clicks "Stop"
graph.cancel(run_id)
```

---

### **6. Cancel vs Interrupt**

| Feature            | Cancel              | Interrupt             |
| ------------------ | ------------------- | --------------------- |
| Purpose            | Terminate execution | Pause for human input |
| Reversible         | ❌ No                | ✔ Yes                 |
| State preserved    | ✔ Checkpointed      | ✔ Active              |
| Continue execution | ❌                   | ✔                     |

---

### **7. Production Use Cases**

| Scenario             | Why Cancel           |
| -------------------- | -------------------- |
| User sends new query | Old run must stop    |
| Loop detected        | Prevent runaway cost |
| Tool failure         | Abort and recover    |
| Policy violation     | Immediate shutdown   |
| System overload      | Free resources       |

---

### **8. Safety Guarantees**

| Guarantee            | Explanation               |
| -------------------- | ------------------------- |
| No partial writes    | State frozen after cancel |
| Tool cleanup         | Open resources closed     |
| Checkpoint integrity | Resume debugging later    |
| Idempotent           | Multiple cancels safe     |

---

### **9. Cancellation + Checkpointing**

After cancellation:

```python
last_state = checkpoint_store.load(run_id)
```

You can **inspect**, **debug**, or **resume** safely.

---

### **10. Design Best Practices**

* Always combine **cancel + timeout**
* Attach **cancel handlers** to critical tools
* Log all cancel events
* Expose cancel control in user interfaces
* Apply rate-based auto cancellation

---

### **11. Mental Model**

> **Cancel = emergency brake for intelligent systems**

It guarantees that **no execution can escape control**, regardless of graph complexity or agent autonomy.

### Demonstration

In [1]:
import asyncio
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    count: int

async def long_task(state):
    await asyncio.sleep(3)       # Simulate slow tool / LLM
    return {"count": state["count"] + 1}

def router(state):
    if state["count"] >= 100:    # Intentionally unreachable
        return END
    return "work"

builder = StateGraph(State)
builder.add_node("work", long_task)
builder.set_entry_point("work")
builder.add_conditional_edges("work", router, {"work": "work", END: END})

graph = builder.compile()


In [2]:
run = graph.astream({"count": 0})

In [7]:
await asyncio.sleep(7)   # Let it run for a few cycles
await run.aclose()        # Cancel the task
try:
	async for _ in run:
		pass
except asyncio.CancelledError:
	print("Graph execution cancelled successfully")